#  Problem (unicode1)

 (a) What Unicode character does chr(0) return?

In [ ]:
chr(0)

 (b) How does this character’s string representation (`__repr__()`) differ from its printed representation?

In [ ]:
print('hello' + chr(0) + 'world')

In [ ]:
repr('hello' + chr(0) + 'world')

打印以可读性为目标展示最终输出，会有不可见字符；`repr()`使用无歧义的转义字符以进行调试和理解等！

(c) What happens when this character occurs in text? It may be helpful to play around with the following in your Python interpreter and see if it matches your expectations:

In [ ]:
print(chr(0))
print('hello' + chr(0) + 'world')

#  Problem (unicode2)

**Unicode Encoding**:  it’s impractical to train tokenizers directly on Unicode codepoints, since the vocabulary would be prohibitively large (around 150K items) and sparse (since many characters are quite rare)

In [ ]:
# Example of UTF-8 encode
test_string = 'hello! こんにちは!'
utf8_encode = test_string.encode('utf-8')
print(utf8_encode)
print(type(utf8_encode)) # 'bytes': 8字节2进制数据 不可变
utf8_encode_list = list(utf8_encode)
print(utf8_encode_list)

# UTF-8为可变长编码方法
print(f'List length: {len(utf8_encode_list)}')
print(f'String length: {len(test_string)}')

print(utf8_encode.decode('utf-8'))

In [ ]:
# Example of UTF-16 encode
test_string = 'hello! こんにちは!'
utf16_encode = test_string.encode('utf-16')
print(utf16_encode)
print(type(utf16_encode))
utf16_encode_list = list(utf16_encode)
print(utf16_encode_list)

# UTF-16为可变长编码方法
print(f'List length: {len(utf16_encode_list)}')
print(f'String length: {len(test_string)}')

print(utf16_encode.decode('utf-16'))

(a) What are some reasons to prefer training our tokenizer on UTF-8 encoded bytes, rather than UTF-16 or UTF-32? It may be helpful to compare the output of these encodings for various input strings.

1. UTF-8 空间最高效
2. UTF-8 最为通用
3. UTF-16/32 需要额外的BOM标记符

(b) Consider the following (incorrect) function, which is intended to decode a UTF-8 byte string into a Unicode string. Why is this function incorrect? Provide an example of an input byte string that yields incorrect results.

(c) Give a two byte sequence that does not decode to any Unicode character(s).

In [ ]:
# 只对1byte字符有效
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])
print(decode_utf8_bytes_to_str_wrong("hello".encode("utf-8")))
try:
    print(decode_utf8_bytes_to_str_wrong("草".encode("utf-8")))
except:
    print(f'an incorrect example: 草')

In [ ]:
# pre-tokenizer for fast merge compute
import regex as re
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
re.findall(PAT, "some text that i'll pre-tokenize")

In [ ]:
# Just test my implementation (fast enough)
from cs336_basics.Tokenizer.tokenizer import run_train_bpe

vocab, merges = run_train_bpe(
    'data/TinyStoriesV2-GPT4-train.txt',
    32000,
    ['<|endoftext|>'],
    **{'n_processes': 8}
)

print(vocab[20000])
print(merges[:10])

In [ ]:
# train and save (1) vocabulary (2) merges (3) encoding over openweb
import base64
import json
from cs336_basics.Tokenizer.tokenizer import run_train_bpe, Tokenizer
import numpy as np

# train vocab and merges
vocab, merges = run_train_bpe(
    'data/TinyStoriesV2-GPT4-train.txt',
    10000,
    ['<|endoftext|>'],
    **{'n_processes': 8}
)

# save vocab and merges
# without loss encode
vocab = {idx: base64.b64encode(token).decode('ascii')  for idx, token in vocab.items()}
with open('data/TinyStoriesV2-GPT4-vocab.json', 'w', encoding = 'utf-8') as f:
    json.dump(vocab, f, ensure_ascii=False, indent = 2)

with open('data/TinyStoriesV2-GPT4-merges.txt', 'w', encoding = 'utf-8') as f:
    for pair in merges:
        first = base64.b64encode(pair[0]).decode('ascii')
        second = base64.b64encode(pair[1]).decode('ascii')
        f.write(f'{first} {second}\n')


# load vocab and merges
with open('data/TinyStoriesV2-GPT4-vocab.json', 'r', encoding = 'utf-8') as f:
    vocab = json.load(f)
vocab = {int(idx): base64.b64decode(encode_b) for idx, encode_b in vocab.items()}

merges = []
with open('data/TinyStoriesV2-GPT4-merges.txt', 'r', encoding = 'utf-8') as f:
    for row in f:
        if not row:
            continue
        parts = row.split()
        if len(parts) == 2:
            first = base64.b64decode(parts[0])
            second = base64.b64decode(parts[1])
            merges.append((first, second))

tkn = Tokenizer(
    vocab, merges, ['<|endoftext|>']
)

# encode the owt train/valid within block
for stage in ['train', 'valid']:
    encodes = []
    with open(f'data/TinyStoriesV2-GPT4-{stage}.txt', 'r', encoding = 'utf-8') as f:
        for line in f:
            if not line:
                continue
            else:
                encodes.extend(tkn.encode(line))
    encodes = np.array(encodes, dtype = np.uint16)
    np.save(f'data/TinyStoriesV2-GPT4-{stage}.npy', encodes)

In [ ]:
# train and save (1) vocabulary (2) merges (3) encoding over openweb
import base64
import json
from cs336_basics.Tokenizer.tokenizer import run_train_bpe, Tokenizer
from multiprocessing import Pool
import numpy as np

# # train vocab and merges
# vocab, merges = run_train_bpe(
#     'data/owt_train.txt',
#     32000,
#     ['<|endoftext|>'],
#     **{'n_processes': 8}
# )

# # save vocab and merges
# # without loss encode
# vocab = {idx: base64.b64encode(token).decode('ascii')  for idx, token in vocab.items()}
# with open('data/owt-vocab.json', 'w', encoding = 'utf-8') as f:
#     json.dump(vocab, f, ensure_ascii=False, indent = 2)

# with open('data/owt-merges.txt', 'w', encoding = 'utf-8') as f:
#     for pair in merges:
#         first = base64.b64encode(pair[0]).decode('ascii')
#         second = base64.b64encode(pair[1]).decode('ascii')
#         f.write(f'{first} {second}\n')


# load vocab and merges
with open('data/owt_vocab.json', 'r', encoding = 'utf-8') as f:
    vocab = json.load(f)
vocab = {int(idx): base64.b64decode(encode_b) for idx, encode_b in vocab.items()}

merges = []
with open('data/owt_merges.txt', 'r', encoding = 'utf-8') as f:
    for row in f:
        if not row:
            continue
        parts = row.split()
        if len(parts) == 2:
            first = base64.b64decode(parts[0])
            second = base64.b64decode(parts[1])
            merges.append((first, second))

tkn = Tokenizer(
    vocab, merges, ['<|endoftext|>']
)

def encode_chunk(chunk_lines):
    """
    对一部分文本行（一个数据块）进行编码。
    这个函数将在每个子进程中运行。
    """
    # 重新初始化分词器或确保分词器可以被子进程安全使用
    # 在这个例子中，tkn 对象是只读的，可以被子进程安全地继承
    encoded_tokens = []
    for line in chunk_lines:
        if line:
            encoded_tokens.extend(tkn.encode(line))
    return encoded_tokens

def parallel_encode_file(input_path, output_path, chunk_size=10000):
    """
    并行读取和编码一个大文件。
    :param input_path: 输入的 txt 文件路径
    :param output_path: 输出的 npy 文件路径
    :param chunk_size: 每个并行任务处理的行数
    """
    lines_chunk = []
    all_encoded_tokens = []
    
    # 使用与BPE训练时相同的进程数，或者根据CPU核心数决定
    num_processes = 8 # 或者 cpu_count()

    with open(input_path, 'r', encoding='utf-8') as f, Pool(processes=num_processes) as pool:
        results = []
        for line in f:
            lines_chunk.append(line)
            if len(lines_chunk) == chunk_size:
                # 当达到块大小时，将任务提交给进程池
                results.append(pool.apply_async(encode_chunk, (lines_chunk,)))
                lines_chunk = []
        
        # 处理剩余的行
        if lines_chunk:
            results.append(pool.apply_async(encode_chunk, (lines_chunk,)))

        # 收集所有并行任务的结果
        for res in results:
            all_encoded_tokens.extend(res.get())

    # 转换为 NumPy 数组并保存
    encodes = np.array(all_encoded_tokens, dtype=np.uint16)
    np.save(output_path, encodes)
    print(f"Successfully encoded and saved to {output_path}")


# 对训练集和验证集进行并行编码
for stage in ['train', 'valid']:
    input_file = f'data/owt_{stage}.txt'
    output_file = f'data/owt-{stage}.npy'
    parallel_encode_file(input_file, output_file)

# # encode the owt train/valid within block
# for stage in ['train', 'valid']:
#     encodes = []
#     with open(f'data/owt_{stage}.txt', 'r', encoding = 'utf-8') as f:
#         for line in f:
#             if not line:
#                 continue
#             else:
#                 encodes.extend(tkn.encode(line))
#     encodes = np.array(encodes, dtype = np.uint16)
#     np.save(f'data/owt-{stage}.npy', encodes)